In [111]:
import credentials

from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Pinecone
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader
from langchain.prompts import PromptTemplate

from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

import pinecone

import time
import os

# Experimentation

This notebook serves as an experimentation on the components involved in the completion of the objective: Generate text in the author's style of writing, making use of stored document information and the knowledge of the trained LLM model. 

The variable breakdown of the experiment is as follows: 

### Constant Variables
- Documents Uploaded (Documents contained are authored only by the intented mimic author)
    - Character Growth Manifesto
    - Tracking
    - Form Follows Function
    - Thriving Romantically
- Queries Asked (Targeting a specific section of document generation)


### Independent Variables
- Embedding Chunk Size
- Embedding Overlap
- Query Search Method *
- Q&A LLM model
- Q&A Cohesion Method *
- Generative Model Query
- Temperature

### Dependent Variables
- Generative output \
**Note:** Each output will have a stored set of variables describing the experimental parameters that produced that output. 

## Simple Generative Case

Create a simple example generative model ahead of creating experiment

In [53]:
INDEX_NAME = "flourishing-humanity"
DIMENSION = 768
CHUNK_SIZE = 1000
CHUNK_OVERLAP = 0
EMBEDDINGS = HuggingFaceEmbeddings()
QA_MODEL = "gpt-3.5-turbo"
TEMPERATURE = 0
QUERY = "Further expand on the information in the character growth manifesto to empower readers. Specifically, generate a daily routine to help them take control of their lives."

In [16]:
credentials.set_credentials()

In [17]:
pinecone.init(os.getenv("PINECONE_API_KEY"), environment=os.getenv("PINECONE_ENV"))

if INDEX_NAME not in pinecone.list_indexes():
    pinecone.create_index(name=INDEX_NAME, dimension=DIMENSION)
    
index = pinecone.Index(INDEX_NAME)

In [18]:
def load_documents(file_path: str):
    loader = DirectoryLoader(file_path, 
                             glob="**/*.txt", 
                             loader_cls=TextLoader, 
                             show_progress=True)  # Directory uploader for txt documents
    documents = loader.load()  # Load documents
    return documents

In [19]:
path = "../kb/"
documents = load_documents(path)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 1434.07it/s]


In [20]:
text_splitter = CharacterTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)
docs = text_splitter.split_documents(documents)

Created a chunk of size 1608, which is longer than the specified 1000
Created a chunk of size 1014, which is longer than the specified 1000
Created a chunk of size 1089, which is longer than the specified 1000
Created a chunk of size 1045, which is longer than the specified 1000
Created a chunk of size 1273, which is longer than the specified 1000
Created a chunk of size 1262, which is longer than the specified 1000
Created a chunk of size 2742, which is longer than the specified 1000
Created a chunk of size 1144, which is longer than the specified 1000
Created a chunk of size 1267, which is longer than the specified 1000
Created a chunk of size 1043, which is longer than the specified 1000
Created a chunk of size 1340, which is longer than the specified 1000
Created a chunk of size 2111, which is longer than the specified 1000
Created a chunk of size 1568, which is longer than the specified 1000
Created a chunk of size 1930, which is longer than the specified 1000
Created a chunk of s

In [9]:
pinecone.init(
    api_key=os.getenv("PINECONE_API_KEY"),  # find at app.pinecone.io
    environment=os.getenv("PINECONE_ENV"),  # next to api key in console
)

In [21]:
vectordb = Pinecone.from_documents(docs, EMBEDDINGS, index_name=INDEX_NAME)

In [22]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [23]:
llm = ChatOpenAI(model_name=QA_MODEL, temperature=TEMPERATURE)

In [25]:
qa = ConversationalRetrievalChain.from_llm(llm, retriever=vectordb.as_retriever(), memory=memory)

In [54]:
result = qa({"question": QUERY})

In [55]:
print(result["answer"])

The Character Growth Manifesto does not specifically outline a daily routine for exercise. However, based on the context provided, it suggests that incorporating strength training 2 to 3 times a week, endurance training 2 to 3 times a week, and daily walking for 45 minutes or 10,000 steps can provide a good balance of exercise. This routine can be modified to fit individual preferences and schedules. It is important to prioritize and find a balance that works for you in order to achieve the desired physical and mental benefits of exercise.


In [59]:
pinecone.delete_index(INDEX_NAME)

## Method

In [118]:
INDEX_NAME = "flourishing-humanity"

In [119]:
QUERY = "Further expand on the information in the character growth manifesto to empower readers. Specifically, generate a rough daily routine to help them take control of their lives."

In [120]:
credentials.set_credentials()

In [121]:
def create_index(index_name: str, dimension: int):
    pinecone.init(os.getenv("PINECONE_API_KEY"), environment=os.getenv("PINECONE_ENV"))

    if index_name not in pinecone.list_indexes():
        pinecone.create_index(name=index_name, dimension=dimension)

    index = pinecone.Index(index_name)
    return index

In [122]:
def load_documents(file_path: str):
    loader = DirectoryLoader(file_path, 
                             glob="**/*.txt", 
                             loader_cls=TextLoader, 
                             show_progress=True)  # Directory uploader for txt documents
    documents = loader.load()  # Load documents
    return documents


def prep_docs(documents, chunk_size, chunk_overlap):
    text_splitter = CharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    docs = text_splitter.split_documents(documents)
    return docs

In [123]:
def initialize_pinecone():
    pinecone.init(
        api_key=os.getenv("PINECONE_API_KEY"),  # find at app.pinecone.io
        environment=os.getenv("PINECONE_ENV"),  # next to api key in console
    )

In [132]:
def experiment_exe(query: str, index_name: str, dimension: int, chunk_size: int, chunk_overlap: int, embedding, model: str, temperature: float):
    index = create_index(index_name, dimension)   # Create index
    
    path = "../kb/"
    documents = load_documents(path)
    docs = prep_docs(documents, chunk_size, chunk_overlap)
    
    initialize_pinecone()
    
    vectordb = Pinecone.from_documents(docs, embedding, index_name=index_name)
    memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
    
    llm = ChatOpenAI(model_name=model, temperature=temperature)
    
    qa = ConversationalRetrievalChain.from_llm(llm, retriever=vectordb.as_retriever(), memory=memory)
    result = qa({"question": QUERY})

    pinecone.delete_index(index_name)  # Delete index for the creation of the next
    return result['answer']

In [133]:
result = experiment_exe(query=QUERY, 
                   index_name=INDEX_NAME, 
                   dimension=768,
                   chunk_size=500,
                   chunk_overlap=0,
                   embedding=HuggingFaceEmbeddings(),
                   model="gpt-3.5-turbo",
                   temperature=0)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 2320.82it/s]
Created a chunk of size 1608, which is longer than the specified 500
Created a chunk of size 1014, which is longer than the specified 500
Created a chunk of size 841, which is longer than the specified 500
Created a chunk of size 953, which is longer than the specified 500
Created a chunk of size 898, which is longer than the specified 500
Created a chunk of size 1089, which is longer than the specified 500
Created a chunk of size 892, which is longer than the specified 500
Created a chunk of size 765, which is longer than the specified 500
Created a chunk of size 1045, which is longer than the specified 500
Created a chunk of size 978, which is longer than the specified 500
Created a chunk of size 647, which is longer than the specified

Created a chunk of size 542, which is longer than the specified 500
Created a chunk of size 504, which is longer than the specified 500
Created a chunk of size 1113, which is longer than the specified 500
Created a chunk of size 687, which is longer than the specified 500
Created a chunk of size 578, which is longer than the specified 500
Created a chunk of size 689, which is longer than the specified 500
Created a chunk of size 861, which is longer than the specified 500
Created a chunk of size 666, which is longer than the specified 500
Created a chunk of size 607, which is longer than the specified 500
Created a chunk of size 504, which is longer than the specified 500
Created a chunk of size 522, which is longer than the specified 500
Created a chunk of size 525, which is longer than the specified 500
Created a chunk of size 1516, which is longer than the specified 500
Created a chunk of size 843, which is longer than the specified 500
Created a chunk of size 599, which is longer t

Created a chunk of size 1021, which is longer than the specified 500
Created a chunk of size 563, which is longer than the specified 500
Created a chunk of size 687, which is longer than the specified 500
Created a chunk of size 555, which is longer than the specified 500
Created a chunk of size 1437, which is longer than the specified 500
Created a chunk of size 638, which is longer than the specified 500
Created a chunk of size 866, which is longer than the specified 500
Created a chunk of size 2527, which is longer than the specified 500
Created a chunk of size 605, which is longer than the specified 500
Created a chunk of size 682, which is longer than the specified 500
Created a chunk of size 575, which is longer than the specified 500
Created a chunk of size 508, which is longer than the specified 500
Created a chunk of size 550, which is longer than the specified 500
Created a chunk of size 882, which is longer than the specified 500
Created a chunk of size 1280, which is longer

In [134]:
print(result)

The Character Growth Manifesto empowers readers by providing them with actionable strategies and practical advice to take control of their lives and create lasting, positive change. While the manifesto does not provide a specific daily routine, it offers a systematic approach to character growth that can be incorporated into a daily routine. Here is a rough outline of a daily routine that can help readers on their journey to self-discovery and personal growth:

1. Morning Reflection: Start your day by reflecting on your character development goals. Take a few moments to remind yourself of your top 3 character traits that you are focusing on cultivating. Visualize yourself embodying these traits throughout the day.

2. Mindfulness Practice: Engage in a mindfulness practice such as meditation or deep breathing exercises. This can help you cultivate self-awareness, manage stress, and stay focused on your character growth journey.

3. Intention Setting: Set clear intentions for the day ahe

## Experiment Variables
Generate the sets of experiments to be executed, from the specified experimental parameters

In [96]:
chunk_sizes = [50, 100, 200, 500, 1000]
chunk_overlaps = [0, 50, 100, 200]
embeddings = [HuggingFaceEmbeddings()]
dimensions = [756]
models = ["gpt-3.5-turbo"]
temperatures = [0, 0.2, 0.5, 0.8, 1.0]

In [97]:
experiments_list = []

for chunk_size in chunk_sizes:
    for chunk_overlap in chunk_overlaps:
        for embedding, dimension in zip(embeddings, dimensions):
            for model in models:
                for temperature in temperatures:
                    exp_var = {'chunk_size': chunk_size,
                               'chunk_overlap': chunk_overlap,
                               'embedding': embedding,
                               'dimension': dimension,
                               'model': model,
                               'temperature': temperature}
                    experiments_list.append(exp_var)       

In [98]:
experiments_list[0]

{'chunk_size': 50,
 'chunk_overlap': 0,
 'embedding': HuggingFaceEmbeddings(client=SentenceTransformer(
   (0): Transformer({'max_seq_length': 384, 'do_lower_case': False}) with Transformer model: MPNetModel 
   (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
   (2): Normalize()
 ), model_name='sentence-transformers/all-mpnet-base-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}),
 'dimension': 756,
 'model': 'gpt-3.5-turbo',
 'temperature': 0}

## Perform Experiment

In [112]:
experiment_template = "Experiment_"
counter = 0

In [113]:
for experiment in experiments_list[:3]:
    # Execute experiment
    result = experiment_exe(query=QUERY, 
                    index_name=INDEX_NAME, 
                   dimension=experiment['dimension'],
                   chunk_size=experiment['chunk_size'],
                   chunk_overlap=experiment['chunk_overlap'],
                   embedding=experiment['embedding'],
                   model=experiment['model'],
                   temperature=experiment['temperature'])
    
    # Experiment name and counting
    experiment_name = experiment_template + str(counter)
    print(experiment_name)
    print(result)
    print("------------------------------------------------------------------------------------------------")
    counter = counter + 1
    time.sleep(1)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 2941.31it/s]
Created a chunk of size 1608, which is longer than the specified 50
Created a chunk of size 133, which is longer than the specified 50
Created a chunk of size 326, which is longer than the specified 50
Created a chunk of size 1014, which is longer than the specified 50
Created a chunk of size 841, which is longer than the specified 50
Created a chunk of size 953, which is longer than the specified 50
Created a chunk of size 481, which is longer than the specified 50
Created a chunk of size 898, which is longer than the specified 50
Created a chunk of size 216, which is longer than the specified 50
Created a chunk of size 1089, which is longer than the specified 50
Created a chunk of size 892, which is longer than the specified 50
Created

Created a chunk of size 1727, which is longer than the specified 50
Created a chunk of size 702, which is longer than the specified 50
Created a chunk of size 484, which is longer than the specified 50
Created a chunk of size 235, which is longer than the specified 50
Created a chunk of size 618, which is longer than the specified 50
Created a chunk of size 1433, which is longer than the specified 50
Created a chunk of size 432, which is longer than the specified 50
Created a chunk of size 588, which is longer than the specified 50
Created a chunk of size 765, which is longer than the specified 50
Created a chunk of size 250, which is longer than the specified 50
Created a chunk of size 655, which is longer than the specified 50
Created a chunk of size 144, which is longer than the specified 50
Created a chunk of size 510, which is longer than the specified 50
Created a chunk of size 280, which is longer than the specified 50
Created a chunk of size 180, which is longer than the specif

Created a chunk of size 341, which is longer than the specified 50
Created a chunk of size 522, which is longer than the specified 50
Created a chunk of size 525, which is longer than the specified 50
Created a chunk of size 1516, which is longer than the specified 50
Created a chunk of size 462, which is longer than the specified 50
Created a chunk of size 468, which is longer than the specified 50
Created a chunk of size 843, which is longer than the specified 50
Created a chunk of size 599, which is longer than the specified 50
Created a chunk of size 857, which is longer than the specified 50
Created a chunk of size 212, which is longer than the specified 50
Created a chunk of size 88, which is longer than the specified 50
Created a chunk of size 1907, which is longer than the specified 50
Created a chunk of size 482, which is longer than the specified 50
Created a chunk of size 600, which is longer than the specified 50
Created a chunk of size 189, which is longer than the specifi

Created a chunk of size 196, which is longer than the specified 50
Created a chunk of size 367, which is longer than the specified 50
Created a chunk of size 874, which is longer than the specified 50
Created a chunk of size 522, which is longer than the specified 50
Created a chunk of size 315, which is longer than the specified 50
Created a chunk of size 527, which is longer than the specified 50
Created a chunk of size 360, which is longer than the specified 50
Created a chunk of size 561, which is longer than the specified 50
Created a chunk of size 626, which is longer than the specified 50
Created a chunk of size 1595, which is longer than the specified 50
Created a chunk of size 577, which is longer than the specified 50
Created a chunk of size 956, which is longer than the specified 50
Created a chunk of size 290, which is longer than the specified 50
Created a chunk of size 241, which is longer than the specified 50
Created a chunk of size 330, which is longer than the specifi

Created a chunk of size 675, which is longer than the specified 50
Created a chunk of size 381, which is longer than the specified 50
Created a chunk of size 370, which is longer than the specified 50
Created a chunk of size 719, which is longer than the specified 50
Created a chunk of size 587, which is longer than the specified 50
Created a chunk of size 268, which is longer than the specified 50
Created a chunk of size 301, which is longer than the specified 50
Created a chunk of size 287, which is longer than the specified 50
Created a chunk of size 765, which is longer than the specified 50
Created a chunk of size 299, which is longer than the specified 50
Created a chunk of size 683, which is longer than the specified 50
Created a chunk of size 82, which is longer than the specified 50
Created a chunk of size 106, which is longer than the specified 50
Created a chunk of size 92, which is longer than the specified 50
Created a chunk of size 113, which is longer than the specified 

...

Created a chunk of size 341, which is longer than the specified 50
Created a chunk of size 522, which is longer than the specified 50
Created a chunk of size 525, which is longer than the specified 50
Created a chunk of size 1516, which is longer than the specified 50
Created a chunk of size 462, which is longer than the specified 50
Created a chunk of size 468, which is longer than the specified 50
Created a chunk of size 843, which is longer than the specified 50
Created a chunk of size 599, which is longer than the specified 50
Created a chunk of size 857, which is longer than the specified 50
Created a chunk of size 212, which is longer than the specified 50
Created a chunk of size 88, which is longer than the specified 50
Created a chunk of size 1907, which is longer than the specified 50
Created a chunk of size 482, which is longer than the specified 50
Created a chunk of size 600, which is longer than the specified 50
Created a chunk of size 189, which is longer than the specifi

Created a chunk of size 196, which is longer than the specified 50
Created a chunk of size 367, which is longer than the specified 50
Created a chunk of size 874, which is longer than the specified 50
Created a chunk of size 522, which is longer than the specified 50
Created a chunk of size 315, which is longer than the specified 50
Created a chunk of size 527, which is longer than the specified 50
Created a chunk of size 360, which is longer than the specified 50
Created a chunk of size 561, which is longer than the specified 50
Created a chunk of size 626, which is longer than the specified 50
Created a chunk of size 1595, which is longer than the specified 50
Created a chunk of size 577, which is longer than the specified 50
Created a chunk of size 956, which is longer than the specified 50
Created a chunk of size 290, which is longer than the specified 50
Created a chunk of size 241, which is longer than the specified 50
Created a chunk of size 330, which is longer than the specifi

Created a chunk of size 675, which is longer than the specified 50
Created a chunk of size 381, which is longer than the specified 50
Created a chunk of size 370, which is longer than the specified 50
Created a chunk of size 719, which is longer than the specified 50
Created a chunk of size 587, which is longer than the specified 50
Created a chunk of size 268, which is longer than the specified 50
Created a chunk of size 301, which is longer than the specified 50
Created a chunk of size 287, which is longer than the specified 50
Created a chunk of size 765, which is longer than the specified 50
Created a chunk of size 299, which is longer than the specified 50
Created a chunk of size 683, which is longer than the specified 50
Created a chunk of size 82, which is longer than the specified 50
Created a chunk of size 106, which is longer than the specified 50
Created a chunk of size 92, which is longer than the specified 50
Created a chunk of size 113, which is longer than the specified 

Created a chunk of size 757, which is longer than the specified 50
Created a chunk of size 386, which is longer than the specified 50
Created a chunk of size 171, which is longer than the specified 50
Created a chunk of size 721, which is longer than the specified 50


Experiment_2
I'm sorry, but I don't have access to the specific information in The Character Growth Manifesto or the ability to generate a daily routine. It would be best to refer to the manifesto itself or any accompanying materials for guidance on creating a daily routine to take control of one's life.
------------------------------------------------------------------------------------------------
